# More Readable Messages with Lookout for Metrics and Lambda

This notebook will show how to create a Lambda function to receive your alerts for L4M and then better pass them on with SNS.


## Sample

Behold the Sample Object Sent to a Lambda:

```json
{
  "alertName": "CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2",
  "alertEventId": "arn:aws:lookoutmetrics:us-east-1:246405784697:Alert:CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2:event/3bf090ef-5437-4262-920c-512d311c1503",
  "anomalyDetectorArn": "arn:aws:lookoutmetrics:us-east-1:246405784697:AnomalyDetector:CanaryAD-BackTest-PT1H-RCF-LOW_VOL-12768b4f-8636-4089-b",
  "alertArn": "arn:aws:lookoutmetrics:us-east-1:246405784697:Alert:CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2",
  "alertDescription": null,
  "impactedMetric": {
    "metricName": "revenue",
    "dimensionContribution": [
      {
        "dimensionName": "colorCode",
        "dimensionValueContributions": [
          {
            "dimensionValue": "blue",
            "valueContribution": 100
          }
        ]
      },
      {
        "dimensionName": "deviceType",
        "dimensionValueContributions": [
          {
            "dimensionValue": "tablet",
            "valueContribution": 100
          }
        ]
      },
      {
        "dimensionName": "marketplace",
        "dimensionValueContributions": [
          {
            "dimensionValue": "CANADA",
            "valueContribution": 100
          }
        ]
      },
      {
        "dimensionName": "platform",
        "dimensionValueContributions": [
          {
            "dimensionValue": "windows",
            "valueContribution": 100
          }
        ]
      }
    ],
    "relevantTimeSeries": [
      {
        "timeSeriesId": "04c628cd2f2d71dde555db5ffd3a98251d2fb027f1a88718bbf94efe26e22ed9f2e5fa9e25367f179ffe4457d38c4153f1c111bde700213d631f1684cfba59b9",
        "dimensions": [
          {
            "dimensionName": "colorCode",
            "dimensionValue": "blue"
          },
          {
            "dimensionName": "deviceType",
            "dimensionValue": "tablet"
          },
          {
            "dimensionName": "marketplace",
            "dimensionValue": "CANADA"
          },
          {
            "dimensionName": "platform",
            "dimensionValue": "windows"
          }
        ],
        "metricValue": 721501.084
      }
    ]
  },
  "timestamp": "2021-02-17T11:00Z[UTC]",
  "anomalyScore": 28.29
}
```

A problem with this is that it is a native JSON string, and we need to read that in for Python, so lets save it as a file "input.json"

In [1]:
# Read in the file as JSON
import json
input_js = json.loads(open('input.json').read())

In [2]:
input_js

{'alertName': 'CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2',
 'alertEventId': 'arn:aws:lookoutmetrics:us-east-1:246405784697:Alert:CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2:event/3bf090ef-5437-4262-920c-512d311c1503',
 'anomalyDetectorArn': 'arn:aws:lookoutmetrics:us-east-1:246405784697:AnomalyDetector:CanaryAD-BackTest-PT1H-RCF-LOW_VOL-12768b4f-8636-4089-b',
 'alertArn': 'arn:aws:lookoutmetrics:us-east-1:246405784697:Alert:CanaryAlert-009a9443-d34a-41af-b782-f6e409db55c2',
 'alertDescription': None,
 'impactedMetric': {'metricName': 'revenue',
  'dimensionContribution': [{'dimensionName': 'colorCode',
    'dimensionValueContributions': [{'dimensionValue': 'blue',
      'valueContribution': 100}]},
   {'dimensionName': 'deviceType',
    'dimensionValueContributions': [{'dimensionValue': 'tablet',
      'valueContribution': 100}]},
   {'dimensionName': 'marketplace',
    'dimensionValueContributions': [{'dimensionValue': 'CANADA',
      'valueContribution': 100}]},
   {'dime

In this more condensed version we can see there are a few things that need to be outlined in the alert:

1. What has happened?
2. Why has it happened?
3. WHen did it happen?
4. Who?(Less important here)
5. How? also less important 

```
That gives us something like this:

An anomaly in METRIC name has been reported at TIMESTAMP.

The anomaly impacted the following dimensions(DIMENSION_LIST).

The anomaly score that was reported was SCORE.

To Learn more information visit the Lookouk for Metrics Console here: CONSOLE_LINK
```

In [27]:
import datetime
from dateutil.parser import parse

In [125]:
def create_anomaly_string(input_json):
    response = ""
    timestamp = parse(input_json['timestamp'], fuzzy_with_tokens=True)[0]
    timestamp1 = timestamp.strftime("%B %d %Y")
    timestamp2 = timestamp.strftime("%H:%M")
    # Setup the time
    response += "An anomaly in " + str(input_json['impactedMetric']['metricName']) + " on " + timestamp1 + ' at ' + timestamp2 + ".\n"
    # Now define your information 
    num_of_time_series = len(input_json['impactedMetric']['relevantTimeSeries'])
    response += str(num_of_time_series) + " time series was impacted.\n\n"
    
    for ts in range(num_of_time_series):
        response += "TimeSeries: " + str(num_of_time_series) + " was impacted on the following dimensions and values:\n"
        for item in input_json['impactedMetric']['relevantTimeSeries'][ts]['dimensions']:
            response += "\t" + item['dimensionName'] + "-" + item['dimensionValue'] + "\n"
    
    response += "\nThe Anomaly  score was: " + str(input_json['anomalyScore']) + "\nn"
    url = "https://us-west-2.console.aws.amazon.com/lookoutmetrics/home?region=us-west-2#"
    url += str(input_json['anomalyDetectorArn'])
    response += "To learn more visit the Lookout for Metrics console at: " + url + " .\n"
 
    return response

In [126]:
response = create_anomaly_string(input_json=input_js)
print(response)

An anomaly in revenue on February 17 2021 at 11:00.
1 time series was impacted.

TimeSeries: 1 was impacted on the following dimensions and values:
	colorCode-blue
	deviceType-tablet
	marketplace-CANADA
	platform-windows

The Anomaly  score was: 28.29
nTo learn more visit the Lookout for Metrics console at: https://us-west-2.console.aws.amazon.com/lookoutmetrics/home?region=us-west-2#arn:aws:lookoutmetrics:us-east-1:246405784697:AnomalyDetector:CanaryAD-BackTest-PT1H-RCF-LOW_VOL-12768b4f-8636-4089-b .

